import library

In [2]:
import numpy as np
import torch
import torch.utils.model_zoo as model_zoo
import torch.nn as nn
from torch.nn import Parameter, DataParallel
import torch.nn.functional as F
from torch.utils.data import Dataset, DataLoader
from torchvision import transforms as T
import os
from PIL import Image
import pandas as pd
import math

build dataset

In [9]:
# buid the dataframe from the the path
identity_root = 'lfw_funneled'
identity_list = os.listdir(identity_root)
#only folder is identity
identity_list = [identity for identity in identity_list if os.path.isdir(os.path.join(identity_root, identity))]
path_list = []
identity_label_list = []
for identity in identity_list:
    image_list = os.listdir(os.path.join(identity_root, identity))
    for img in image_list:
        path = os.path.join(identity_root, identity, img)
        path_list.append(path)
        identity_label_list.append(identity)

In [15]:
data_df = pd.DataFrame({'path': path_list, 'identity': identity_label_list})
data_df['identity_code'] = pd.Categorical(data_df['identity']).codes
data_df['identity_code'] = data_df['identity_code'].astype('int32')

In [34]:
pairing_file = 'lfw_test_pair.txt'
root = 'lfw_funneled'
f = open(pairing_file, 'r')
imgs = f.readlines()
imgs = [os.path.join(root, img[:-1]) for img in imgs] # remove \n

img_paths = []
identity = []
for img in imgs:
    img_split = img.split()
    img_paths.append(img_split[0])
    identity.append(img_split[1].split('/')[0])
data_df = pd.DataFrame({'img_path': img_paths, 'identity': identity}) # to df
# endcode identity
data_df['identity_code'] = pd.Categorical(data_df['identity']).codes
data_df['identity_code'] = data_df['identity_code'].astype('int32')
data_df.head()

,img_path,identity,identity_code
0,lfw_funneled\Abel_Pacheco/Abel_Pacheco_0001.jpg,Abel_Pacheco,6
1,lfw_funneled\Akhmed_Zakayev/Akhmed_Zakayev_000...,Akhmed_Zakayev,20
2,lfw_funneled\Akhmed_Zakayev/Akhmed_Zakayev_000...,Akhmed_Zakayev,20
3,lfw_funneled\Amber_Tamblyn/Amber_Tamblyn_0001.jpg,Amber_Tamblyn,61
4,lfw_funneled\Anders_Fogh_Rasmussen/Anders_Fogh...,Anders_Fogh_Rasmussen,71


In [35]:
class FaceDataset(Dataset):
    def __init__(self, data_df, input_shape, phase="train"):
        self.path = data_df['img_path'].values
        self.label = data_df['identity_code'].values
        self.phase = phase
        self.input_shape = input_shape
        if self.phase == 'train':
            self.transforms = T.Compose([
                T.RandomCrop(self.input_shape[1:]),
                T.RandomHorizontalFlip(),
                T.ToTensor(),
                T.Normalize(mean=[0.5], std=[0.5])
            ])
        else:   
            self.transforms = T.Compose([
                T.CenterCrop(self.input_shape[1:]),
                T.ToTensor(),
                T.Normalize(mean=[0.5], std=[0.5])
            ])
            
    def __len__(self):
        return len(self.label)
    
    def __getitem__(self, index):
        path = self.path[index]
        label = self.label[index]
        data = Image.open(path)
        data = data.convert('L')
        data = self.transforms(data)
        return data.float(), label

In [36]:
# dataloader
data_df = data_df.copy()
input_shape = (1, 250, 250)
phase = 'train'
dataset = FaceDataset(data_df, input_shape, phase)
dataloader = DataLoader(dataset, batch_size=32, shuffle=True)

In [37]:
num_classes = len(np.unique(data_df['identity'].values))
num_classes

3149

In [38]:
for data, label in dataloader:
    print(data.shape)
    print(label.shape)
    break

torch.Size([32, 1, 250, 250])
torch.Size([32])


build model

In [40]:
# load pretrained model
model = torch.hub.load('pytorch/vision:v0.6.0', 'resnet18', weights=True)
# change first and last layer
model.conv1 = nn.Conv2d(1, 64, kernel_size=7, stride=2, padding=3, bias=False)
model.fc = nn.Linear(512, 512)

Using cache found in C:\Users\earth/.cache\torch\hub\pytorch_vision_v0.6.0
C:\Users\earth\AppData\Roaming\Python\Python310\site-packages\torchvision\models\_utils.py:223: UserWarning: Arguments other than a weight enum or `None` for 'weights' are deprecated since 0.13 and may be removed in the future. The current behavior is equivalent to passing `weights=ResNet18_Weights.IMAGENET1K_V1`. You can also use `weights=ResNet18_Weights.DEFAULT` to get the most up-to-date weights.
  warnings.warn(msg)


In [41]:
model

ResNet(
  (conv1): Conv2d(1, 64, kernel_size=(7, 7), stride=(2, 2), padding=(3, 3), bias=False)
  (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
  (relu): ReLU(inplace=True)
  (maxpool): MaxPool2d(kernel_size=3, stride=2, padding=1, dilation=1, ceil_mode=False)
  (layer1): Sequential(
    (0): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
      (conv2): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn2): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
    )
    (1): BasicBlock(
      (conv1): Conv2d(64, 64, kernel_size=(3, 3), stride=(1, 1), padding=(1, 1), bias=False)
      (bn1): BatchNorm2d(64, eps=1e-05, momentum=0.1, affine=True, track_running_stats=True)
      (relu): ReLU(inplace=True)
  

In [28]:
model_urls = {
    'resnet18': 'https://download.pytorch.org/models/resnet18-5c106cde.pth',
    'resnet34': 'https://download.pytorch.org/models/resnet34-333f7ec4.pth',
    'resnet50': 'https://download.pytorch.org/models/resnet50-19c8e357.pth',
    'resnet101': 'https://download.pytorch.org/models/resnet101-5d3b4d8f.pth',
    'resnet152': 'https://download.pytorch.org/models/resnet152-b121ed2d.pth',
}

In [29]:
def conv3x3(in_planes, out_planes, stride=1):
    """3x3 convolution with padding"""
    return nn.Conv2d(in_planes, out_planes, kernel_size=3, stride=stride,
                     padding=1, bias=False)

class BasicBlock(nn.Module):
    expansion = 1

    def __init__(self, inplanes, planes, stride=1, downsample=None):
        super(BasicBlock, self).__init__()
        self.conv1 = conv3x3(inplanes, planes, stride)
        self.bn1 = nn.BatchNorm2d(planes)
        self.relu = nn.ReLU(inplace=True)
        self.conv2 = conv3x3(planes, planes)
        self.bn2 = nn.BatchNorm2d(planes)
        self.downsample = downsample
        self.stride = stride

    def forward(self, x):
        residual = x

        out = self.conv1(x)
        out = self.bn1(out)
        out = self.relu(out)

        out = self.conv2(out)
        out = self.bn2(out)

        if self.downsample is not None:
            residual = self.downsample(x)

        out += residual
        out = self.relu(out)

        return out
    
class ResNet(nn.Module):

    def __init__(self, block, layers):
        self.inplanes = 64
        super(ResNet, self).__init__()
        # self.conv1 = nn.Conv2d(3, 64, kernel_size=7, stride=2, padding=3,
        #                        bias=False)
        self.conv1 = nn.Conv2d(1, 64, kernel_size=3, stride=1, padding=1,
                               bias=False)
        self.bn1 = nn.BatchNorm2d(64)
        self.relu = nn.ReLU(inplace=True)
        # self.maxpool = nn.MaxPool2d(kernel_size=3, stride=2, padding=1)
        self.layer1 = self._make_layer(block, 64, layers[0], stride=2)
        self.layer2 = self._make_layer(block, 128, layers[1], stride=2)
        self.layer3 = self._make_layer(block, 256, layers[2], stride=2)
        self.layer4 = self._make_layer(block, 512, layers[3], stride=2)
        # self.avgpool = nn.AvgPool2d(8, stride=1)
        # self.fc = nn.Linear(512 * block.expansion, num_classes)
        self.fc5 = nn.Linear(512 * 8 * 8, 512)

        for m in self.modules():
            if isinstance(m, nn.Conv2d):
                nn.init.kaiming_normal_(m.weight, mode='fan_out', nonlinearity='relu')
            elif isinstance(m, nn.BatchNorm2d):
                nn.init.constant_(m.weight, 1)
                nn.init.constant_(m.bias, 0)

    def _make_layer(self, block, planes, blocks, stride=1):
        downsample = None
        if stride != 1 or self.inplanes != planes * block.expansion:
            downsample = nn.Sequential(
                nn.Conv2d(self.inplanes, planes * block.expansion,
                          kernel_size=1, stride=stride, bias=False),
                nn.BatchNorm2d(planes * block.expansion),
            )

        layers = []
        layers.append(block(self.inplanes, planes, stride, downsample))
        self.inplanes = planes * block.expansion
        for i in range(1, blocks):
            layers.append(block(self.inplanes, planes))

        return nn.Sequential(*layers)

    def forward(self, x):
        x = self.conv1(x)
        x = self.bn1(x)
        x = self.relu(x)
        # x = self.maxpool(x)

        x = self.layer1(x)
        x = self.layer2(x)
        x = self.layer3(x)
        x = self.layer4(x)
        # x = nn.AvgPool2d(kernel_size=x.size()[2:])(x)
        # x = self.avgpool(x)
        x = x.view(x.size(0), -1)
        x = self.fc5(x)

        return x

def resnet18(pretrained=False, **kwargs):
    """Constructs a ResNet-18 model.
    Args:
        pretrained (bool): If True, returns a model pre-trained on ImageNet
    """
    model = ResNet(BasicBlock, [2, 2, 2, 2], **kwargs)
    if pretrained:
        model.load_state_dict(model_zoo.load_url(model_urls['resnet18']))
    return model

In [39]:
# model = resnet18()
# model

loss function

In [42]:
class ArcMarginProduct(nn.Module):
    r"""Implement of large margin arc distance: :
        Args:
            in_features: size of each input sample
            out_features: size of each output sample
            s: norm of input feature
            m: margin

            cos(theta + m)
        """
    def __init__(self, in_features, out_features, s=30.0, m=0.50, easy_margin=False):
        super(ArcMarginProduct, self).__init__()
        self.in_features = in_features
        self.out_features = out_features
        self.s = s
        self.m = m
        self.weight = Parameter(torch.FloatTensor(out_features, in_features))
        nn.init.xavier_uniform_(self.weight)

        self.easy_margin = easy_margin
        self.cos_m = math.cos(m)
        self.sin_m = math.sin(m)
        self.th = math.cos(math.pi - m)
        self.mm = math.sin(math.pi - m) * m

    def forward(self, input, label):
        # --------------------------- cos(theta) & phi(theta) ---------------------------
        cosine = F.linear(F.normalize(input), F.normalize(self.weight))
        sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
        phi = cosine * self.cos_m - sine * self.sin_m
        if self.easy_margin:
            phi = torch.where(cosine > 0, phi, cosine)
        else:
            phi = torch.where(cosine > self.th, phi, cosine - self.mm)
        # --------------------------- convert label to one-hot ---------------------------
        # one_hot = torch.zeros(cosine.size(), requires_grad=True, device='cuda')
        one_hot = torch.zeros(cosine.size(), device='cuda')
        one_hot.scatter_(1, label.view(-1, 1).long(), 1)
        # -------------torch.where(out_i = {x_i if condition_i else y_i) -------------
        output = (one_hot * phi) + ((1.0 - one_hot) * cosine)  # you can use torch.where if your torch.__version__ is 0.4
        output *= self.s
        # print(output)

        return output

train

In [48]:
data_df = data_df.copy()    
num_classes = len(np.unique(data_df['identity'].values))
model = DataParallel(model)
metric_fc = DataParallel(ArcMarginProduct(512, num_classes, s=30, m=0.5, easy_margin=False))
criterion = nn.CrossEntropyLoss()
optimizer = torch.optim.Adam([{'params': model.parameters()}, {'params': metric_fc.parameters()}], lr=0.001)
model.train()
for data, label in dataloader:
    feature = model(data)
    output = metric_fc(feature, label)
    loss = criterion(output, label.long().cuda())
    # optimizer.zero_grad()
    # loss.backward()
    # optimizer.step()
    break


In [229]:
input = feature
label = label.cuda()
in_features = 512
out_features = num_classes
s = 30.0
m = 0.5
easy_margin = False
weight = Parameter(torch.FloatTensor(out_features, in_features))
# xavier
weight = nn.init.xavier_uniform_(weight)
cos_m = math.cos(m)
sin_m = math.sin(m)
th = math.cos(math.pi - m)
mm = math.sin(math.pi - m) * m

In [230]:
weight.shape

torch.Size([3149, 512])

In [231]:
cosine = F.linear(F.normalize(input), F.normalize(weight.to('cuda')))
sine = torch.sqrt((1.0 - torch.pow(cosine, 2)).clamp(0, 1))
phi = cosine * cos_m - sine * sin_m
phi

tensor([[-0.5148, -0.4559, -0.5472,  ..., -0.4524, -0.4593, -0.4947],
        [-0.4979, -0.4467, -0.5516,  ..., -0.4365, -0.4535, -0.5010],
        [-0.5183, -0.5058, -0.5171,  ..., -0.4462, -0.4597, -0.5097],
        ...,
        [-0.4651, -0.4422, -0.5457,  ..., -0.4557, -0.3954, -0.5223],
        [-0.5026, -0.5000, -0.5428,  ..., -0.4734, -0.4254, -0.5356],
        [-0.5294, -0.4540, -0.5094,  ..., -0.4866, -0.4513, -0.5328]],
       device='cuda:0', grad_fn=<SubBackward0>)

In [232]:
one_hot = torch.zeros(cosine.size(), device='cuda')
one_hot.scatter_(1, label.view(-1, 1).long(), 1)

tensor([[0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        ...,
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.],
        [0., 0., 0.,  ..., 0., 0., 0.]], device='cuda:0')

In [233]:
output = (one_hot * phi) + ((1.0 - one_hot) * cosine)
output *= s

In [234]:
output

tensor([[-1.2237,  0.7968, -2.3694,  ...,  0.9153,  0.6833, -0.5244],
        [-0.6351,  1.1089, -2.5271,  ...,  1.4494,  0.8776, -0.7430],
        [-1.3441, -0.9090, -1.3034,  ...,  1.1257,  0.6686, -1.0445],
        ...,
        [ 0.4866,  1.2567, -2.3144,  ...,  0.8054,  2.8000, -1.4843],
        [-0.7989, -0.7075, -2.2110,  ...,  0.2046,  1.8177, -1.9559],
        [-1.7375,  0.8609, -1.0339,  ..., -0.2463,  0.9532, -1.8545]],
       device='cuda:0', grad_fn=<MulBackward0>)